# Unit test of different function


In [ ]:
import json as js

In [16]:
#download the data and create a json file 
!curl -X GET https://api.capio.ai/v1/speech/transcript/593f237fbcae700012ba8fcd --header "apiKey: 262ac9a0c9ba4d179aad4c0b9b02120a" > capio.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60146  100 60146    0     0  60146      0  0:00:01 --:--:--  0:00:01  100k


In [57]:
#read data
with open('capio.json', 'rt') as f:
    data = f.read()

In [82]:
#json decode data 
capio_data=js.loads(data)

In [64]:
#hava a look at specific format 
capio_data[0]["result"][0]["alternative"][0]["words"]


[{'confidence': 1,
  'from': 0.6899999976158142,
  'to': 1.2899999618530273,
  'word': 'um'}]

# Data Prepocessing part


In [8]:
#data-prepocessing steps
def transfer_data(alist):
    all_info=[]
    for i in range(0,len(alist)):
        all_info.append(alist[i]["result"][0]["alternative"][0]["words"])
    return all_info

In [66]:
clean_data=transfer_data(capio_data)

In [67]:
clean_data[2]

[{'confidence': 1,
  'from': 9.710000038146973,
  'to': 9.860000610351562,
  'word': 'they'},
 {'confidence': 1,
  'from': 9.860000610351562,
  'to': 10.039999961853027,
  'word': "don't"},
 {'confidence': 1,
  'from': 10.039999961853027,
  'to': 10.280000686645508,
  'word': 'learn'},
 {'confidence': 1,
  'from': 10.280000686645508,
  'to': 10.730000495910645,
  'word': 'enough'}]

In [9]:
#construct the new data structures according to the clean data because we will let some words become red
def construct_sentence(alist):
    whole_sentence=[]
    for i in range(0,len(alist)):
        new_list=[]    
        word_list=[]
        red_list=[]   #have low confidence
        super_red_list=[] #have lower confidence
        for j in range(0,len(alist[i])):
            word_info=alist[i][j]
            word=word_info["word"]
            if word_info["confidence"]<=0.75 and word_info["confidence"]>=0.7:
                red_list.append(j)
            if word_info["confidence"]<0.7:
                super_red_list.append(j)
            word_list.append(word)
        new_list.append("Begin stamp is: "+str(alist[i][0]["from"]))
        new_list.append(word_list)
        new_list.append(red_list)
        new_list.append(super_red_list)
        whole_sentence.append(new_list)
    return whole_sentence

In [92]:
capio_data=construct_sentence(clean_data)

In [93]:
capio_data[0] #the data structure is [time,[word],[red_word],[super_red_word]]

['Begin stamp is: 0.6899999976158142', ['um'], [], []]

In [95]:
capio_data[0]

['Begin stamp is: 0.6899999976158142', ['um'], [], []]

# Write_to_Docx function

In [11]:
from docx import Document
from docx.shared import Inches
from docx.shared import RGBColor
def write_doc(data,astring):
    document = Document()
    document.add_heading("Capio Python Code Challenge", 0)
    for i in range(0,len(data)):
        sentence=data[i]
        time=sentence[0]
        words=sentence[1]
        red_list=sentence[2]
        super_red_list=sentence[3]

        #if the confidence is between 0.7 and 0.75, we will use erythrine
        #if the confidence is lower than 0.7, we will use rose-red color 
        p=document.add_paragraph(time+", Transcript result: ")
        p.add_run(" ")
        for j in range(len(words)):
            if j not in red_list and j not in super_red_list:
                p.add_run(words[j])
            else:
                if j in red_list:
                    red_word=p.add_run(words[j])
                    red_word.font.color.rgb=RGBColor(255,127,80)
                else:
                    red_word=p.add_run(words[j])
                    red_word.font.color.rgb=RGBColor(255,0,0)
            p.add_run(" ")
    document.save(astring)

In [102]:
write_doc(capio_data,"capio_data_challenge.docx")

# Main function


In [17]:

def main():
    with open('capio.json', 'rt') as f:
        data = f.read()
    capio_data=js.loads(data)
    clean_data=transfer_data(capio_data)
    capio_data=construct_sentence(clean_data)
    write_doc(capio_data,"capio_data_challenge.docx")
if __name__=="__main__":
    main()

# Test Part if json file is not empty 

In [80]:
#create some easy json file but with low confidence,union test

In [83]:
test=result="""[  
  {  
    "result":[  
      {  
        "alternative":[  
          {  
            "confidence":1,
            "transcript":"how do i get a passport",
            "words":[  
              {  
                "confidence":0.42,
                "from":0.02,
                "to":0.14,
                "word":"how"
              },
              {  
                "confidence":0.62,
                "from":0.14,
                "to":0.27,
                "word":"do"
              },
              {  
                "confidence":0.53,
                "from":0.27,
                "to":0.32,
                "word":"i"
              },
              {  
                "confidence":0.99,
                "from":0.32,
                "to":0.49,
                "word":"get"
              },
              {  
                "confidence":0.72,
                "from":0.49,
                "to":0.54,
                "word":"a"
              },
              {  
                "confidence":0.99,
                "from":0.54,
                "to":1.25,
                "word":"passport"
              }
            ]
          }
        ],
        "final":true
      }
    ],
    "result_index":0
  }
]""";

In [84]:
test_data=js.loads(test)

In [86]:
test_clean_data=transfer_data(test_data)

In [87]:
test_final_data=construct_sentence(test_clean_data)

In [103]:
write_doc(test_final_data,"test_result.docx")

# Test Part Two-if the json file is empty

In [5]:
test_2="""[]
""";

In [3]:
import json as js

In [6]:
test2_data=js.loads(test_2)

In [12]:
test_2_clean_data=transfer_data(test2_data)

In [13]:
test2_final_data=construct_sentence(test_2_clean_data)

In [14]:
write_doc(test2_final_data,"test2_result.docx")